#CAD PROJECT

Baseline model based on:
https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#initialize-and-reshape-the-networks


Made by MAIA team: Jaime, Ahmed & "Prem"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


* In **finetuning**, we start with a pretrained model and update *all* of the model’s parameters for our new task, in essence retraining the whole model.
* In **feature extraction** we start with a pretrained model and only update the final layer weights from which we derive predictions. It is called feature extraction because we use the pretrained CNN as a fixed feature-extractor, and only change the output layer.

In general both transfer learning methods follow the same few steps:

-  Initialize the pretrained model
-  Reshape the final layer(s) to have the same number of outputs as the
   number of classes in the new dataset
-  Define for the optimization algorithm which parameters we want to
   update during training
-  Run the training step




In [2]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import random
import pandas as pd
import sys
import os
import copy
%matplotlib inline

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

!pip install barbar

PyTorch Version:  1.3.1
Torchvision Version:  0.4.2


In [0]:
path_folder='/content/drive/My Drive/Colab Notebooks/CAD Project/'
sys.path.insert(1, path_folder) # Import py files in the folder

Inputs
------


The other inputs are as follows: ``num_classes`` is the number of
classes in the dataset, ``batch_size`` is the batch size used for
training and may be adjusted according to the capability of your
machine, ``num_epochs`` is the number of training epochs we want to run,
and ``feature_extract`` is a boolean that defines if we are finetuning
or feature extracting. If ``feature_extract = False``, the model is
finetuned and all model parameters are updated. If
``feature_extract = True``, only the last layer parameters are updated,
the others remain fixed.




In [6]:
###### SELECT CHALLANGE TYPE ##########

CHALLANGE_TYPE = {
    1: "Dermo Challenge",
    2: "Histopathology Challenge",
}
CHALLANGE_TYPE_INDEX = 2;

CHALLANGE_TYPE_STR =CHALLANGE_TYPE.get(CHALLANGE_TYPE_INDEX, "Invalid normalization type")
print(CHALLANGE_TYPE_STR)

if(CHALLANGE_TYPE_INDEX == 1):
  INPUT_SIZE=225
if(CHALLANGE_TYPE_INDEX == 2):
  INPUT_SIZE=96

Histopathology Challenge


In [0]:
options = {}
options["input_size"]=INPUT_SIZE
options["challenge_dir"]=os.path.join(path_folder,CHALLANGE_TYPE_STR+"/")
if(CHALLANGE_TYPE_INDEX == 1):
  options["experiments_dir"]=os.path.join(path_folder,"Dermo_Experiments")
if(CHALLANGE_TYPE_INDEX == 2):
  options["experiments_dir"]=os.path.join(path_folder,"Histo_Experiments")

Initialize and Reshape the Networks
-----------------------------------



#Test Image

In [0]:
ensemble_architectures=["densenet","resnext50"] 
ensemble_models={"densenet_17":2,
                 "resnext50_13":1,}

In [19]:
from MAIA_model import initialize_model

import csv
from MAIA_Loader import get_val_loader

test_scans = os.listdir(options['test_dir'])
test_scans.sort()
test_scans.sort(key=len)
m = nn.Softmax(dim=1)

if(CHALLANGE_TYPE_INDEX == 1):
  test_result_path = '/content/drive/My Drive/Colab Notebooks/CAD Project/Dermo_Experiments/dermo_test.csv'

if(CHALLANGE_TYPE_INDEX == 2):
  test_result_path ='/content/drive/My Drive/Colab Notebooks/CAD Project/Histo_Experiments/histo_test.csv'

options["test_dir"]=os.path.join(options["challenge_dir"],"test/")

with open(test_result_path, 'w') as csvfile:
  filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
  for i, scan_name in enumerate(test_scans):
    scan_path = os.path.join(options['test_dir'], scan_name)
    print(scan_path)
    input_image = get_test_loader(scan_path,options["input_size"])
    total_prob=np.zeros((1,2))
    #Ensemble Models
    for arch, model in zip(ensemble_architectures,ensemble_models):
      model_ft= initialize_model(arch, num_classes, FEATURE_EXTRACT, use_pretrained=False)
      model_ft.load_state_dict(torch.load(options["experiments_dir"]+"/Train_complete/"+model +'.pt'))
      model_ft.to('cuda')
      # Model evaluation 
      model_ft.eval()
      outputs = model_ft(input_image)
      outputs = m(outputs)
      #print(outputs)
      total_prob=total_prob+ outputs.cpu().detach().numpy()*ensemble_models[model] 
          
    preds = np.argmax(total_prob, 1)
    #print(preds)
    filewriter.writerow(str(preds))

/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage0.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage1.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage2.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage3.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage4.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage5.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage6.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage7.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage8.png
/content/drive/My Drive/Colab Notebooks/CAD Project/Histopathology Challenge/test/testImage9.png
/content/drive/My Drive/Colab 